Note that PDFs are already downloaded via the Conservation Evidence website.

In [5]:
# get all pdf files in subdirectories, but not at top level
def get_pdf_files(directory):
    import os
    pdf_files = []
    for root, dirs, files in os.walk(directory):
        if root == directory: # don't include files I'm not sure about
            continue
        for file in files:
            if file.endswith(".pdf"):
                pdf_files.append(os.path.join(root, file).replace("\\", "/"))
                # pdf_files.append(os.path.join(root, file))
    return pdf_files

filepaths = get_pdf_files('../../../data/synopses/')
filepaths

['../../../data/synopses/Amphibians/Amphibian.pdf',
 '../../../data/synopses/Animals Ex-Situ/Management of Captive Animals.pdf',
 '../../../data/synopses/Bats/Bat.pdf',
 '../../../data/synopses/Birds/Bird.pdf',
 '../../../data/synopses/Farmland/Farmland.pdf',
 '../../../data/synopses/Farmland/Mediterranean Farmland.pdf',
 '../../../data/synopses/Farmland/Soil Fertility.pdf',
 '../../../data/synopses/Farmland/Sustainable Farming.pdf',
 '../../../data/synopses/Fish/Marine Fish.pdf',
 '../../../data/synopses/Forests/Forest.pdf',
 '../../../data/synopses/Grassland/Grassland.pdf',
 '../../../data/synopses/Insects/Bee.pdf',
 '../../../data/synopses/Insects/Butterfly and Moth.pdf',
 '../../../data/synopses/Invasive Fish/Invasive Freshwater Species.pdf',
 '../../../data/synopses/Mammals/Primate.pdf',
 '../../../data/synopses/Mammals/Terrestrial Mammal.pdf',
 '../../../data/synopses/Marine/Biodiversity of Marine Artificial Structures.pdf',
 '../../../data/synopses/Marine/Sustainable Aquaculture

In [6]:
def populate_json(result):
    # result has the following keys: id, title, author, year, text, classes

    json = {
        "reference type": "CE Study",
        "author": result["author"],
        "year": result["year"],
        "title": result["title"],
        "series editor": None,
        "series/book title": None,
        "place published": "None",
        "institution": "Conservation Evidence Team",
        "publisher": "Conservation Evidence",
        "date": None,
        "doi": None,
        "class": max(
            set(result["classes"]), key=result["classes"].count, default=None
        ),
        "abstract": None,
        "url": 'PDF-PLACEHOLDER',
        "text": result["text"],
    }
    return json

In [7]:
import pymupdf

def parse_pdf(pdf_path):
    pdf = pymupdf.open(pdf_path)
    text = ""
    for page in pdf:
        text += page.get_text()


    author = pdf.metadata.get("Author", None)
    year = pdf.metadata.get("ModDate", None) # convert from bash to year
    if year is not None:
        year = year[2:6]
    title = pdf.metadata.get("Title", None)
    if title is None:
        title = pdf_path.split("/")[-1].split(".")[0]

    folder = pdf_path.split("/")[2]

    pdf.close()
    return populate_json({
        "author": author,
        "year": year,
        "title": title,
        "text": text,
        "classes": [folder],
    })

In [8]:
from tqdm.notebook import tqdm

def write_json(i,record):

    import json
    with open(f'../../../data/labelled/synopses/{i} - {record["title"]}.json', 'w') as f:
        json.dump(record, f)

for i,filepath in enumerate(tqdm(filepaths)):
    record = parse_pdf(filepath)
    write_json(i,record)


  0%|          | 0/24 [00:00<?, ?it/s]